In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import os
from google.colab import files
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN

import logging

#from tensorflow.keras import layers


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
csv_path = "/content/drive/MyDrive/Data-Glacier/Week 12/Data/newForecast.csv"
forecast = pd.read_csv(csv_path)
forecast.head()

,Unnamed: 0,Product,date,Sales,Discount,In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS,DayOfWeek,Quarter,WeekNum
0,0,SKU1,2017-02-05,27750,0.00,0,0,0,0.0,0,0,0,0,Sunday,1,5
1,1,SKU1,2017-02-12,29023,0.00,1,0,1,0.0,0,1,0,0,Sunday,1,6
2,2,SKU1,2017-02-19,45630,0.17,0,0,0,0.0,0,0,0,0,Sunday,1,7
3,3,SKU1,2017-02-26,26789,0.00,1,0,1,0.0,0,0,0,0,Sunday,1,8
4,4,SKU1,2017-03-05,41999,0.17,0,0,0,0.0,0,0,0,0,Sunday,1,9


In [4]:
#change date column to date time and drop the unnamed column
forecast['date'] = pd.to_datetime(forecast['date'])
forecast.drop('Unnamed: 0', axis=1, inplace=True)

#add year and month
forecast['Year'] = forecast['date'].dt.year
forecast['Month'] = forecast['date'].dt.month
#forecast['Day'] = forecast['date'].dt.day

#drop DayOfWeek column
forecast = forecast.drop(columns=['DayOfWeek'])

#forecast.head()

In [5]:
forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Product          1218 non-null   object        
 1   date             1218 non-null   datetime64[ns]
 2   Sales            1218 non-null   int64         
 3   Discount         1218 non-null   float64       
 4   In-Store Promo   1218 non-null   int64         
 5   Catalogue Promo  1218 non-null   int64         
 6   Store End Promo  1218 non-null   int64         
 7   Google_Mobility  1218 non-null   float64       
 8   Covid_Flag       1218 non-null   int64         
 9   V_DAY            1218 non-null   int64         
 10  EASTER           1218 non-null   int64         
 11  CHRISTMAS        1218 non-null   int64         
 12  Quarter          1218 non-null   int64         
 13  WeekNum          1218 non-null   int64         
 14  Year             1218 non-null   int64  

In [6]:
#seperate by product

product1 = forecast[forecast["Product"] == "SKU1"]
product2 = forecast[forecast["Product"] == "SKU2"]
product3 = forecast[forecast["Product"] == "SKU3"]
product4 = forecast[forecast["Product"] == "SKU4"]
product5 = forecast[forecast["Product"] == "SKU5"]
product6 = forecast[forecast["Product"] == "SKU6"]

In [7]:
product1 = product1.set_index("date")
product1.head(3)

,Product,Sales,Discount,In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS,Quarter,WeekNum,Year,Month
date,,,,,,,,,,,,,,,
2017-02-05,SKU1,27750,0.00,0,0,0,0.0,0,0,0,0,1,5,2017,2
2017-02-12,SKU1,29023,0.00,1,0,1,0.0,0,1,0,0,1,6,2017,2
2017-02-19,SKU1,45630,0.17,0,0,0,0.0,0,0,0,0,1,7,2017,2


In [8]:
dates = product1.index

#Split targets and features
X = product1.iloc[:, 2:14]
#X = product1[["In-Store Promo",	"Catalogue Promo", "Store End Promo", "Quarter", "WeekNum", "Year", "Month"]]
y = product1["Sales"]

In [9]:
#product1.iloc[:, 2:15]
#df.iloc[:, np.r_[2, 5:10]]

In [10]:
from sklearn.model_selection import train_test_split

#Split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

#train and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
    test_size=0.2, random_state=0)

#Count features for modelization
X_num_columns= len(X.columns)

# Standardize

In [11]:
#standardize dataset
mean = np.mean(X_train)
stddev = np.std(X_train)

#standardize mean
X_train = (X_train - mean) / stddev
X_test = (X_test - mean) / stddev

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


# Training

In [12]:
EPOCHS = 15
BATCH_SIZE = 16

model = Sequential()
model.add(SimpleRNN(128, activation='relu',
                    input_shape=(None, 1)))

model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error',
              optimizer = 'adam',
              metrics = ['mean_absolute_error'])


model.summary()
history = model.fit(X_train, y_train,
                    validation_data= (X_val, y_val),
                    epochs=EPOCHS,
                    batch_size = BATCH_SIZE,
                    #verbose = 2,
                    shuffle=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               16640     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 16769 (65.50 KB)
Trainable params: 16769 (65.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/15
9/9 [==============================] - 3s 71ms/step - loss: 3529513216.0000 - mean_absolute_error: 49983.6680 - val_loss: 1986583680.0000 - val_mean_absolute_error: 38541.6133
Epoch 2/15
9/9 [==============================] - 0s 15ms/step - loss: 3529210880.0000 - mean_absolute_error: 49980.9375 - val_loss: 1977360640.0000 - val_mean_absolute_error: 38440.6914
Epoch 3/15
9/9 [==========================

# Testing

In [13]:
model2 = Sequential()
model2.add(SimpleRNN(128, activation='relu',
                    input_shape=(None, 1)))

model2.add(Dense(1, activation='linear'))
model2.compile(loss='mean_squared_error',
              optimizer = 'adam',
              metrics = ['mean_absolute_error'])


model2.summary()
history = model2.fit(X_test, y_test,
                    validation_data= (X_train, y_train),
                    epochs=EPOCHS,
                    batch_size = BATCH_SIZE,
                    #verbose = 2,
                    shuffle=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 128)               16640     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 16769 (65.50 KB)
Trainable params: 16769 (65.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/15
3/3 [==============================] - 1s 141ms/step - loss: 2665729024.0000 - mean_absolute_error: 46959.9531 - val_loss: 3529522432.0000 - val_mean_absolute_error: 49983.7266
Epoch 2/15
3/3 [==============================] - 0s 30ms/step - loss: 2665703680.0000 - mean_absolute_error: 46959.7109 - val_loss: 3529493504.0000 - val_mean_absolute_error: 49983.4531
Epoch 3/15
3/3 [=======================